# Warmup Exercise #
First lets import the necessary libraries:

In [1]:
from basics import asm_interp

Now let's define some helper functions for the first exersice to help us with the StASM code.


In [2]:
def STORE(addr, value):
    """
    Store value in addr.
    We will use this function to test our implementation
    """
    return [
        ("PUSH", value),
        ("PUSH", addr),
        ("POP", 2),
        ("STOR", 0)
    ]

def READ_FROM_ARRAY():
    """
    Pops the top as index and then base addr and push the value at that index from the array at addr
    :return:
    """
    return [
        ("POP", 2),  # r0 = index, r1 = base addr
        ("ALU", "ADD"),  # push base addr + index
        ("POP", 1),   # r0 = base addr + index
        ("LOAD", 0),  # push value at base addr + index
    ]

Comment: 
Since I didn't find in the PDF what is the convention for calling a function and returning a value, I decided to use the common convention of returning the value via some register (I picked r1) and for calling a function we pass the args on the stack from first to last.
for example:
```
fuction foo(a, b, c):
    # some logic
    return d
```
The stack for calling foo should look like:
[a, b, c]
(The stack grows this way -->)
and the ret value from foo is saved at r1,
so at the end of foo in our example the stack will look like:
[]
and register r1 contains d.


## First Exercise ##
For the first exercise, the program is:
```
for (at = 0; at < n; at++) {
    if (a[at] == v) break;
}
return at;
```


As I said before, we assume we get the functions arg on the stack, so the stack looks like this:
[base_addr, n, v]

The StASM code will be:

In [3]:
program_find_v = [
    ("PUSH", 0x0),  # int i = 0
    "CHECK_COND:",
    ("DUP", 0),  # Push i on stack
    ("DUP", 3),  # Push n on stack
    ("POP", 2),  # r0 = i, r1 = n
    ("ALU", "LT"),  # Compare i < n
    ("POP", 1),  # r0 = comparison result
    ("JNZ", "LOOP_BODY"),  # If i < n, jump to loop body
    ("JMP", "NOT_FOUND"),  # Otherwise, exit loop
    "INC_I:",  # Increment i
    ("PUSH", 1),  # Push constant 1
    ("POP", 2),  # r0 = i r1 = 1
    ("ALU", "ADD"),  # Push i + 1
    ("JMP", "CHECK_COND"),  # Jump to check condition
    "LOOP_BODY:",
    ("DUP", 0),  # Push i on stack
    ("DUP", 4),  # Push base_addr on stack
    *READ_FROM_ARRAY(),  # Push a[i]
    ("DUP", 2),
    ("POP", 2),
    ("ALU", "SUB"),
    ("POP", 1),  # r0 = a[i] - v
    ("JZ", "FOUND"),  # If a[i] == v, jump to found
    ("JMP", "INC_I"),  # Otherwise, check condition
    "FOUND:",  # Found the value
    ("POP", 2),  # Emptying the stack
    ("POP", 2),
    ("PUSH", 1),  # Push True
    ("PUSH", 1),  # Push True
    ("JMP", "END"),  # Jump to end
    "NOT_FOUND:",  # Not found
    ("POP", 2),  # Emptying the stack
    ("POP", 2),
    ("PUSH", 0),  # Push False
    ("PUSH", 0),  # Push False
    "END:",
    ("POP", 2)
]

# Tests #
We will define the following function that will help us test the code:

In [4]:
from array import array

def test_vm(pre_stack_layout, pre_mem_layout, program, post_stack_layout, post_mem_layout, return_val):
    """
    This function will test the program by executing it with the given stack and memory layouts 
    and checking if the final stack and memory layouts match the expected ones.
    :param pre_stack_layout: The stack before executing the program
    :param pre_mem_layout: The memory before executing the program
    :param program: The program to execute
    :param post_stack_layout: The expected stack after executing the program
    :param post_mem_layout: The expected memory after executing the program
    :param return_val: The expected return value
    :return: True if the test passed, False otherwise
    """
    SETUP_STACK_COMMANDS = [("PUSH", value) for value in pre_stack_layout]
    SETUP_MEM_COMMANDS = [STORE(i, value) for i, value in enumerate(pre_mem_layout)]
    SETUP_MEM_COMMANDS = [cmd for sublist in SETUP_MEM_COMMANDS for cmd in sublist] # Flatten list of lists
    inter = asm_interp.AsmInterp()
    inter.execute_program(SETUP_STACK_COMMANDS + SETUP_MEM_COMMANDS)
    print(f"State after setup: \n{inter}")
    inter.execute_program(program)
    print("\nTESTS:")
    assert inter.stack == post_stack_layout, f"Expected stack: {post_stack_layout}, but got: {inter.stack}"
    print("✔️ Stack state passed")
    assert inter.memory == array('H', post_mem_layout), f"Expected memory: {post_mem_layout}, but got: {inter.memory.tolist()}"
    print("✔️ Memory state passed")
    assert inter.r1 == return_val, f"Expected return value: {return_val}, but got: {inter.r1}"
    print(f"✔️ Return value is correct (r1 is {return_val})")
    
    print("\nFinal state of VM: \n", inter)
    
    

To test the function we'll first provide some tests:

## Test 1 ##

In [5]:
test_vm(pre_stack_layout=[0, 4, 8],
        pre_mem_layout=[2, 8, 1, 3],
        program=program_find_v, 
        post_stack_layout=[],
        post_mem_layout=[2, 8, 1, 3],
        return_val=1)

State after setup: 
stack: [0, 4, 8]
r0: 3, r1: 3
mem: [2, 8, 1, 3]

TESTS:
✔️ Stack state passed
✔️ Memory state passed
✔️ Return value is correct (r1 is 1)

Final state of VM: 
 stack: []
r0: 1, r1: 1
mem: [2, 8, 1, 3]


Notice that 8 is in the array so the return value is 1 (True).

## Test 2 ##
Now lets test the case where the value is not in the array by changing 8 to 80 at a[1]

In [6]:
test_vm(pre_stack_layout=[0, 4, 8],
        pre_mem_layout=[2, 80, 1, 3],
        program=program_find_v,
        post_stack_layout=[],
        post_mem_layout=[2, 80, 1, 3],
        return_val=0)

State after setup: 
stack: [0, 4, 8]
r0: 3, r1: 3
mem: [2, 80, 1, 3]

TESTS:
✔️ Stack state passed
✔️ Memory state passed
✔️ Return value is correct (r1 is 0)

Final state of VM: 
 stack: []
r0: 0, r1: 0
mem: [2, 80, 1, 3]


As you can see the program returned 0 via register r1, which means False.

## Test 3 ##
Let's test starting address that isn't 0

In [7]:
test_vm(pre_stack_layout=[3, 4, 2],
        pre_mem_layout=[1, 2, 3, 20, 80, 10, 30, 31, 34, 33],
        program=program_find_v,
        post_stack_layout=[],
        post_mem_layout=[1, 2, 3, 20, 80, 10, 30, 31, 34, 33],
        return_val=0)


State after setup: 
stack: [3, 4, 2]
r0: 33, r1: 9
mem: [1, 2, 3, 20, 80, 10, 30, 31, 34, 33]

TESTS:
✔️ Stack state passed
✔️ Memory state passed
✔️ Return value is correct (r1 is 0)

Final state of VM: 
 stack: []
r0: 0, r1: 0
mem: [1, 2, 3, 20, 80, 10, 30, 31, 34, 33]


## Test 4 ##
Same as Test 3 but now the value is inside the array

In [8]:
test_vm(pre_stack_layout=[3, 5, 31],
        pre_mem_layout=[1, 2, 3, 20, 80, 10, 30, 31, 34, 33],
        program=program_find_v,
        post_stack_layout=[],
        post_mem_layout=[1, 2, 3, 20, 80, 10, 30, 31, 34, 33],
        return_val=1)

State after setup: 
stack: [3, 5, 31]
r0: 33, r1: 9
mem: [1, 2, 3, 20, 80, 10, 30, 31, 34, 33]

TESTS:
✔️ Stack state passed
✔️ Memory state passed
✔️ Return value is correct (r1 is 1)

Final state of VM: 
 stack: []
r0: 1, r1: 1
mem: [1, 2, 3, 20, 80, 10, 30, 31, 34, 33]


# Second Exercise #
For the second exercise, finding the max of an array, we assume we get the functions arg on the stack, so the stack looks like this:
[base_addr, n] 

The StASM code will be:

In [9]:
program_find_max = [
    ("PUSH", 0),
    ("DUP", 2),
    *READ_FROM_ARRAY(),
    # Stack is now: [arr_addr, length, a[0]]
    # Now let's define max = a[0]
    ("DUP", 0),
    # Stack is now: [arr_addr, length, a[0], mx=a[0]]
    # Now after we saved a[0] on stack we can use this mem for i
    # lets set i = 1 to memory address &a[0]
    ("PUSH", 1),
    ("DUP", 4),
    ("POP", 2),
    ("STOR", 0),
    "CHECK_COND:",
    # First put i in r0 and n in r1
    ("DUP", 3),
    ("POP", 1),
    ("LOAD", 0),
    ("DUP", 3),
    ("POP", 2),
    # Check i < n
    ("ALU", "LT"),
    ("POP", 1),
    ("JNZ", "LOOP_BODY"),
    ("JMP", "END"),
    "LOOP_BODY:",
    # read i from mem and put on stack
    ("DUP", 3),
    ("POP", 1),
    ("LOAD", 0),
    # put base addr on stack
    ("DUP", 4),
    # Put a[i] on stack
    *READ_FROM_ARRAY(),
    # Put mx on stack
    ("DUP", 1),
    # now stack is [base_addr, n, a[0], mx, a[i], mx]
    ("POP", 2),
    ("ALU", "LT"),
    # now stack is [base_addr, n, a[0], mx, (result a[i] < mx)]
    ("POP", 1),
    ("JNZ", "INC_I"),
    ("JMP", "UPDATE"),
    "INC_I:",
    # Put i on stack
    ("PUSH", 0),
    ("DUP", 4),
    *READ_FROM_ARRAY(),
    ("PUSH", 1),
    ("POP", 2),
    ("ALU", "ADD"),
    ("DUP", 4),
    ("POP", 2),
    ("STOR", 0),
    ("JMP", "CHECK_COND"),
    "UPDATE:",
    ("POP", 1),  # remove old mx from stack
    # Push base addr
    ("DUP", 2),
    ("POP", 1),
    ("LOAD", 0),
    # Now i is on top of stack
    ("DUP", 3),
    *READ_FROM_ARRAY(),  # so now a[i] is where mx was on stack so mx = a[i]
    ("JMP", "INC_I"),
    "END:",
    # Restore a[0]:
    # Put a[0] on stack (we saved it before):
    ("DUP", 1),
    # Put base_addr on stack:
    ("DUP", 4),
    ("POP", 2),
    ("STOR", 0),
    ("POP", 2),
    ("POP", 1),
    ("POP", 1)
]

# Tests #
To test the logic lets put some values on the stack and run the program.

## Test 1 ##

In [10]:
test_vm(pre_stack_layout=[0, 4],
        pre_mem_layout=[2, 8, 10, 3],
        program=program_find_max,
        post_stack_layout=[],
        post_mem_layout=[2, 8, 10, 3],
        return_val=10)


State after setup: 
stack: [0, 4]
r0: 3, r1: 3
mem: [2, 8, 10, 3]

TESTS:
✔️ Stack state passed
✔️ Memory state passed
✔️ Return value is correct (r1 is 10)

Final state of VM: 
 stack: []
r0: 0, r1: 10
mem: [2, 8, 10, 3]


## Test 2 ##
Let's test the case where the address of the array isn't 0

In [11]:
test_vm(pre_stack_layout=[1, 3],
        pre_mem_layout=[300, 1, 2, 3, 300, 300],
        program=program_find_max,
        post_stack_layout=[],
        post_mem_layout=[300, 1, 2, 3, 300, 300],
        return_val=3)



State after setup: 
stack: [1, 3]
r0: 300, r1: 5
mem: [300, 1, 2, 3, 300, 300]

TESTS:
✔️ Stack state passed
✔️ Memory state passed
✔️ Return value is correct (r1 is 3)

Final state of VM: 
 stack: []
r0: 1, r1: 3
mem: [300, 1, 2, 3, 300, 300]
